<a href="https://colab.research.google.com/github/annachrome/Travel_Bookings_Chatbot/blob/main/tools_conversation_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install panel
!pip install amadeus
!pip install wikipedia
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.2/178.2 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for amadeus: filename=amadeus-9.0.0-py2.py3-none-any.whl size=75046 sha256=41eeb4135637b9e08e694042f0e62f536dc69aaf5e026f3871b1cf42b5ae0de1
  Stored in directory: /root/.cache/pip/wheels/b0/50/ea/3417d93eee6760a945d7711333d8d42b9f482e84600ef7f711
Successfully built amadeus
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=cba5af70d0a7b10aa3ac924f2e27742c399db027a4b800d344a5d92b9862270a
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia
     ━━━━━━━━━

In [11]:
# from langchain import OpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.schema.agent import AgentFinish
from langchain.schema.runnable import RunnablePassthrough
from langchain.agents import AgentExecutor


# Tools
from pydantic import BaseModel, Field
from langchain.tools import tool

from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

from langchain.agents.agent_toolkits.amadeus.toolkit import AmadeusToolkit
import requests
from datetime import datetime
import wikipedia

import panel as pn  # GUI
import param
import os
os.environ["OPENAI_API_KEY"] = ''

# Amadeus
toolkit = AmadeusToolkit()
# TODO tools = toolkit.get_tools()
AMADEUS_CLIENT_ID=''
AMADEUS_CLIENT_SECRET=''
os.environ["AMADEUS_CLIENT_ID"] = AMADEUS_CLIENT_ID
os.environ["AMADEUS_CLIENT_SECRET"] = AMADEUS_CLIENT_SECRET

# LangSmith
LANCHAIN_API_KEY=''
os.environ["LANGCHAIN_TRACING_V2"] = "TRUE"
os.environ["LANGCHAIN_PROJECT"] = "gpt4_tools_tutorial"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = LANCHAIN_API_KEY

# Debugging - show inputs/intermediate steps/outpus
# from langchain.globals import set_debug
# set_debug(True)

In [4]:
# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""

    BASE_URL = "https://api.open-meteo.com/v1/forecast"

    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }
    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.utcnow()
    time_list = [datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']

    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]

    return f'The current temperature is {current_temperature}°C'


@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)


@tool
def get_datetime(query: str) -> str:
    """Get current/now datetime."""
    print(type(query))
    return datetime.now().isoformat()


In [8]:
class cbfs(param.Parameterized):

    def __init__(self, tools, **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.functions = [format_tool_to_openai_function(f) for f in tools]
        self.model = ChatOpenAI(temperature=0).bind(functions=self.functions)
        self.memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are helpful travel agent."),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
        ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()
        self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=False, memory=self.memory)

    # def convchain(self, query):
    #     if not query:
    #         return
    #     inp.value = ''
    #     result = self.qa.invoke({"input": query})
    #     self.answer = result['output']
    #     self.panels.extend([
    #         pn.Row('User:', pn.pane.Markdown(query, width=450)),
    #         pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=450, styles={'background-color': '#F6F6F6'}))
    #     ])
    #     return pn.WidgetBox(*self.panels, scroll=True)

    # def clr_history(self,count=0):
    #     self.chat_history = []
    #     return

In [ ]:
tools = [get_current_temperature, search_wikipedia, get_datetime]
cb = cbfs(tools)

In [10]:
result = cb.qa.invoke({"input": "What's the temperature in Amman now"})

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign > 4:chain:RunnableParallel] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign > 4:chain:RunnableParallel > 5:chain:<lambda>] Entering Chain run with input:
[inputs]
[chain/end] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign > 4:chain:RunnableParallel > 5:chain:<lambda>] s] Exiting Chain run with output:
{
  "output": []
}
[chain/end] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign > 4:chain:RunnableParallel] s] Exiting Chain run with output:
{
  "agent_scratch

[llm/end] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 7:llm:ChatOpenAI] [2.46s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "",
        "generation_info": {
          "finish_reason": "function_call"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain_core",
            "messages",
            "ai",
            "AIMessage"
          ],
          "kwargs": {
            "content": "",
            "additional_kwargs": {
              "function_call": {
                "arguments": "{\n  \"latitude\": 31.9566,\n  \"longitude\": 35.9457\n}",
                "name": "get_current_temperature"
              }
            }
          }
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "completion_tokens": 29,
      "prompt_tokens": 198,
      "total_tokens": 227
    },
    "model_name": "gpt-3.5-turbo",
    "system_finger

In [12]:
result = cb.qa.invoke({"input": "How cold will it be there next May?"})

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign > 4:chain:RunnableParallel] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign > 4:chain:RunnableParallel > 5:chain:<lambda>] Entering Chain run with input:
[inputs]
[chain/end] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign > 4:chain:RunnableParallel > 5:chain:<lambda>] s] Exiting Chain run with output:
{
  "output": []
}
[chain/end] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign > 4:chain:RunnableParallel] s] Exiting Chain run with output:
{
  "agent_scratch

In [ ]:
result = cb.qa.invoke({"input": "So what clothes should I pack for my trip there in May"})

In [14]:
result = cb.qa.invoke({"input": "What is there to do in Jordan"})

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign > 4:chain:RunnableParallel] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign > 4:chain:RunnableParallel > 5:chain:<lambda>] Entering Chain run with input:
[inputs]
[chain/end] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign > 4:chain:RunnableParallel > 5:chain:<lambda>] s] Exiting Chain run with output:
{
  "output": []
}
[chain/end] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign > 4:chain:RunnableParallel] s] Exiting Chain run with output:
{
  "agent_scratch

[llm/end] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 7:llm:ChatOpenAI] [1.94s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "",
        "generation_info": {
          "finish_reason": "function_call"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain_core",
            "messages",
            "ai",
            "AIMessage"
          ],
          "kwargs": {
            "content": "",
            "additional_kwargs": {
              "function_call": {
                "arguments": "{\n  \"query\": \"Tourism in Jordan\"\n}",
                "name": "search_wikipedia"
              }
            }
          }
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "completion_tokens": 19,
      "prompt_tokens": 553,
      "total_tokens": 572
    },
    "model_name": "gpt-3.5-turbo",
    "system_fingerprint": null
  },
  "ru

In [15]:
result = cb.qa.invoke({"input": "Plan me a detailed 7-day itinerary that includes your first 5 suggestions"})

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign > 4:chain:RunnableParallel] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign > 4:chain:RunnableParallel > 5:chain:<lambda>] Entering Chain run with input:
[inputs]
[chain/end] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign > 4:chain:RunnableParallel > 5:chain:<lambda>] s] Exiting Chain run with output:
{
  "output": []
}
[chain/end] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign > 4:chain:RunnableParallel] s] Exiting Chain run with output:
{
  "agent_scratch

**Q: what time is it in jordan** - used datetime tool

A: The current time in Jordan is 23:44 (11:44 PM)

**Q: What's the temperature in Amman now** - used weather tool

A: The current temperature in Amman, Jordan is 11.1°C.

**Q: How cold will it be there next May?**

A: I'm sorry, but I don't have access to future weather data. However, in May, the weather in Amman, Jordan is generally warm with temperatures ranging from 18°C to 28°C. It is advisable to check a reliable weather forecast closer to the date for more accurate information.
Human: So what clothes should I pack for my trip there in May

**Q: So what clothes should I pack for my trip there in May**

A: In May, the weather in Amman, Jordan is generally warm and pleasant. Here are some clothing suggestions for your trip:

1. Lightweight and breathable clothing: Pack lightweight and breathable clothing such as t-shirts, shorts, skirts, and dresses to stay comfortable in the warm weather.

2. Light layers: Although the days are warm, the evenings can be cooler. Pack a light jacket or sweater for the evenings or if you plan to visit higher elevations.

3. Sun protection: Don't forget to pack a hat, sunglasses, and sunscreen to protect yourself from the sun's rays.

4. Comfortable walking shoes: Amman is a city with many historical sites and attractions to explore. Make sure to pack comfortable walking shoes or sandals for sightseeing.

5. Modest clothing: Jordan is a conservative country, and it is respectful to dress modestly, especially when visiting religious sites. Pack clothing that covers your shoulders and knees.

6. Swimwear: If you plan to visit resorts or swimming pools, pack swimwear.

Remember to check the weather forecast closer to your travel date for more accurate information and adjust your packing accordingly.

**Q: What is there to do in Jordan.**

A: Jordan offers a variety of attractions and activities for tourists. Here are some highlights:
  
  1. Petra: Petra is a UNESCO World Heritage Site and one of the most famous archaeological sites in the world. It is known for its stunning rock-cut architecture, including the iconic Treasury (Al-Khazneh). Exploring the ancient city of Petra is a must-visit experience.
  
  2. Amman: The capital city of Jordan, Amman, is a vibrant and modern city with a rich history. Visit the Amman Citadel, Roman Theater, and the bustling markets of downtown to experience the city's cultural and historical heritage.
  
  3. Wadi Rum: Wadi Rum is a desert landscape famous for its stunning red sand dunes, towering cliffs, and rock formations. Take a jeep tour, go hiking, or even spend a night in a Bedouin camp to experience the beauty of the desert.
  
  4. Dead Sea: The Dead Sea is a unique natural wonder and the lowest point on Earth. Float effortlessly in its salty waters and indulge in mineral-rich mud treatments known for their therapeutic properties.
  
  5. Jerash: Explore the ancient Roman ruins of Jerash, one of the best-preserved Roman cities in the world. Walk through the colonnaded streets, visit the temples, and imagine life in the Roman Empire.
  
  6. Aqaba: Aqaba is a coastal city located on the Red Sea. It offers beautiful beaches, vibrant coral reefs, and excellent opportunities for snorkeling and scuba diving. Enjoy water sports, relax by the beach, or take a boat trip to explore the marine life.
  
  7. Mount Nebo: Visit Mount Nebo, a significant biblical site where Moses is said to have seen the Promised Land before he died. Enjoy panoramic views of the Jordan Valley, the Dead Sea, and the surrounding landscapes.
  
  8. Madaba: Known as the "City of Mosaics," Madaba is famous for its ancient mosaic art. Visit the Madaba Archaeological Park and the Church of St. George to see intricate mosaic masterpieces.
  
  These are just a few highlights, and there is much more to explore in Jordan. The country also offers opportunities for hiking, cultural experiences, shopping, and delicious cuisine.


**Q: Plan me a detailed 7-day itinerary that includes your first 5 suggestions**

A: Certainly! Here's a detailed 7-day itinerary that includes the suggested attractions:

Day 1: Arrival in Amman
- Arrive in Amman and check into your hotel.
- Spend the day exploring the city's highlights, such as the Amman Citadel and Roman Theater.
- Enjoy a traditional Jordanian dinner at a local restaurant.

Day 2: Petra
- Take an early morning drive to Petra.
- Spend the day exploring the ancient city of Petra, including the Treasury, the Monastery, and the Royal Tombs.
- Enjoy a guided tour or explore the site at your own pace.
- Return to Amman in the evening.

Day 3: Jerash and Ajloun Castle
- Take a day trip to Jerash, known for its well-preserved Roman ruins.
- Explore the ancient city, including the colonnaded streets, temples, and theaters.
- Visit Ajloun Castle, a medieval fortress offering panoramic views of the surrounding area.
- Return to Amman in the evening.

Day 4: Wadi Rum
- Depart from Amman and head to Wadi Rum.
- Take a jeep tour through the stunning desert landscape, visiting iconic sites and enjoying the unique rock formations.
- Experience a traditional Bedouin lunch and learn about their culture.
- Enjoy a magical sunset in the desert.
- Overnight stay in a Bedouin camp.

Day 5: Aqaba and the Dead Sea
- Leave Wadi Rum and head to Aqaba on the Red Sea coast.
- Spend the morning snorkeling or diving in the vibrant coral reefs.
- Enjoy a delicious seafood lunch by the beach.
- In the afternoon, travel to the Dead Sea.
- Float in the salty waters and indulge in a mineral-rich mud treatment.
- Overnight stay at a Dead Sea resort.

Day 6: Mount Nebo and Madaba
- Visit Mount Nebo, where Moses is said to have seen the Promised Land.
- Enjoy panoramic views of the Jordan Valley and the Dead Sea.
- Continue to Madaba and explore the city's mosaic art, including the Madaba Archaeological Park and the Church of St. George.
- Return to Amman in the evening.

Day 7: Departure
- Depending on your flight time, you may have some free time in Amman for shopping or further exploration.
- Depart from Amman with wonderful memories of your Jordanian adventure.

Please note that this itinerary is a suggestion and can be customized based on your preferences and the availability of attractions. It is also recommended to check the opening hours and any travel advisories before visiting the attractions.

In [ ]:
# pn.extension()

# tools = [get_current_temperature, search_wikipedia, get_datetime]
# cb = cbfs(tools)

# inp = pn.widgets.TextInput( placeholder='Enter text here…')

# conversation = pn.bind(cb.convchain, inp)

# tab1 = pn.Column(
#     pn.Row(inp),
#     pn.layout.Divider(),
#     pn.panel(conversation,  loading_indicator=True, height=400),
#     pn.layout.Divider(),
# )

# dashboard = pn.Column(
#     pn.Row(pn.pane.Markdown('# Travel_Agent')),
#     pn.Tabs(('Conversation', tab1))
# )
# dashboard